In [4]:
pip install google-cloud-vision


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, io

from google.cloud import vision
from google.cloud import vision_v1
from google.cloud.vision_v1 import types

In [3]:
#the JSON file you downloaded in step 5 above
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'image-analysis-366800-b03a47707e98.json'

In [4]:
# Instantiates a client
client = vision.ImageAnnotatorClient()

In [1]:
#pip install glob

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [21]:
### Declaring a function to take input image, analyze it and store the results in a pandas dataframe.


import glob
import imageio
import os, io
from PIL import Image
import pandas as pd

def google_api_input(image,df_image,image_path):
    
    
############ OBJECT DETECTION #####################
    response_object = client.object_localization(image=image).localized_object_annotations
    n_objects = 0
    object_name,object_score=[], []


    #print('Number of objects found: {}'.format(len(response_object)))
    n_objects=len(response_object)

    for object_ in response_object:
        #print('\n{} (confidence: {})'.format(object_.name, object_.score))
        object_name.append(object_.name)
        object_score.append(object_.score)
        
#### LABEL DETECTION ######

    response_label = client.label_detection(image=image)
    label_description,label_score = [], []
    for label in response_label.label_annotations:
        #print({'label': label.description, 'score': label.score})
        label_description.append(label.description)
        label_score.append(label.score)
        
#### LOGO DETECTION ######

    response_logo = client.logo_detection(image=image)
    logo_description,logo_score = [], []
    
    for logo in response_logo.logo_annotations:
        #print({'logo': logo.description, 'score': logo.score})
        logo_description.append(logo.description)
        logo_score.append(logo.score)
        
#### FACE DETECTION ######

    response_face = client.face_detection(image=image)

    face_data = []
    emotion_detection = []

    for face_detection in response_face.face_annotations:
        d = {
            'confidence': face_detection.detection_confidence,
            'joy': face_detection.joy_likelihood,
            'sorrow': face_detection.sorrow_likelihood,
            'surprise': face_detection.surprise_likelihood,
            'anger': face_detection.anger_likelihood
        }
        #print(d)
        emotion_detection.append(d)
    
#### IMAGE PROPERTIES ######

    response_image = client.image_properties(image=image)

    image_data = []
    image_color = []
    
    for c in response_image.image_properties_annotation.dominant_colors.colors[:3]:
        d = {
            'color': c.color,
            'score': c.score,
            'pixel_fraction': c.pixel_fraction
        }
        #print(d)
        image_color.append(d)
        
#### BEST LABEL DETECTION ######

    response_best_label = client.web_detection(image=image)
    annotations = response_best_label.web_detection
    best_label = []

    if annotations.best_guess_labels:
        for label in annotations.best_guess_labels:
            #print('\nBest guess label: {}'.format(label.label))
            best_label.append(label.label)
            
#### TEXT DETECTION ######

    response_text = client.text_detection(image=image)
    text_detection = []

    for r in response_text.text_annotations:
        d = {
            'text': r.description
        }
        #print(d)
        text_detection.append(d)
    
    
    df_image = df_image.append({'image_path':image_path,'object_name':object_name,'object_score':object_score,'label_description':label_description,
                   'label_score':label_score,'logo_description':logo_description,'logo_score':logo_score,
                    'emotion_detection':emotion_detection,'image_color':image_color,'best_label':best_label,
                   'text_detection':text_detection},ignore_index=True )
    
    return df_image
 

In [22]:

## Declaring the folder directory and a dataframe to store the results 


folder = "C:\\Users\\smades2\\Downloads\\Image_Captioning\\*.jpg"

df_image=pd.DataFrame(columns=['image_path','object_name','object_score','label_description','label_score','logo_description',
                                    'logo_score','emotion_detection','image_color','best_label','text_detection'])

## Looping through multiple images stored in the specified directory.

for image_path in glob.glob(folder):
    with io.open(image_path, "rb") as image:
        im = image.read()
    image = types.Image(content=im)
    df_image=google_api_input(image,df_image,image_path)


Number of objects found: 6

Pants (confidence: 0.9432424902915955)

Person (confidence: 0.9416975975036621)

Outerwear (confidence: 0.9354179501533508)

Person (confidence: 0.9120193123817444)

Outerwear (confidence: 0.8700735569000244)

Shoe (confidence: 0.5615472197532654)
{'label': 'Shoe', 'score': 0.9464053511619568}
{'label': 'Sleeve', 'score': 0.8724385499954224}
{'label': 'Interior design', 'score': 0.8550720810890198}
{'label': 'Wood', 'score': 0.8282844424247742}
{'label': 'Thigh', 'score': 0.8173187971115112}
{'label': 'Floor', 'score': 0.813668429851532}
{'label': 'Flooring', 'score': 0.809370756149292}
{'label': 'Knee', 'score': 0.7979599833488464}
{'label': 'Fashion design', 'score': 0.7852624654769897}
{'label': 'Eyewear', 'score': 0.7670047283172607}
{'confidence': 0.778600811958313, 'joy': <Likelihood.VERY_LIKELY: 5>, 'sorrow': <Likelihood.VERY_UNLIKELY: 1>, 'surprise': <Likelihood.VERY_UNLIKELY: 1>, 'anger': <Likelihood.VERY_UNLIKELY: 1>}
{'confidence': 0.9665747284889

{'confidence': 0.640642523765564, 'joy': <Likelihood.VERY_UNLIKELY: 1>, 'sorrow': <Likelihood.VERY_UNLIKELY: 1>, 'surprise': <Likelihood.VERY_UNLIKELY: 1>, 'anger': <Likelihood.VERY_UNLIKELY: 1>}
{'color': red: 179
green: 116
blue: 48
, 'score': 0.12916676700115204, 'pixel_fraction': 0.009744898416101933}
{'color': red: 6
green: 4
blue: 9
, 'score': 0.11500127613544464, 'pixel_fraction': 0.11749999970197678}
{'color': red: 197
green: 195
blue: 207
, 'score': 0.09935201704502106, 'pixel_fraction': 0.11142857372760773}

Best guess label: 
{'text': 'IIIII'}
{'text': 'IIIII'}
Number of objects found: 9

Pants (confidence: 0.9430245161056519)

Hat (confidence: 0.904980480670929)

Necklace (confidence: 0.8789346218109131)

Person (confidence: 0.8653624057769775)

Shoe (confidence: 0.8514818549156189)

Glasses (confidence: 0.8348760604858398)

Shoe (confidence: 0.8126330971717834)

Outerwear (confidence: 0.7857125997543335)

Lighting (confidence: 0.5607669949531555)
{'label': 'Shoe', 'score':

In [23]:
## Checking the dataframe generated
df_image

,image_path,object_name,object_score,label_description,label_score,logo_description,logo_score,emotion_detection,image_color,best_label,text_detection
0,C:\Users\smades2\Downloads\Image_Captioning\co...,"[Pants, Person, Outerwear, Person, Outerwear, ...","[0.9432424902915955, 0.9416975975036621, 0.935...","[Shoe, Sleeve, Interior design, Wood, Thigh, F...","[0.9464053511619568, 0.8724385499954224, 0.855...",[],[],"[{'confidence': 0.778600811958313, 'joy': Like...","[{'color': red: 202 green: 196 blue: 196 , 'sc...",[shoulder],[{'text': '00000 00000 AUGLAN DRIP AMERICANO L...
1,C:\Users\smades2\Downloads\Image_Captioning\co...,"[Sunglasses, Coffee cup, Saucer, Person, Neckl...","[0.9149388074874878, 0.8950873613357544, 0.857...","[Cat, Espressino, Tableware, Cortado, Flat whi...","[0.957766592502594, 0.915120542049408, 0.91348...",[Pagani],[0.559945821762085],"[{'confidence': 0.7045764327049255, 'joy': Lik...","[{'color': red: 238 green: 235 blue: 234 , 'sc...",[cat coffee],[{'text': 'CATTIVI PROPOSITI vensio conline co...
2,C:\Users\smades2\Downloads\Image_Captioning\co...,"[Person, Top, Shirt, Lighting, Packaged goods,...","[0.8287495970726013, 0.7886994481086731, 0.656...","[Arm, Shoulder, Product, Sleeve, Eyewear, Gas,...","[0.9458796977996826, 0.9409850835800171, 0.907...",[Vans],[0.8396409153938293],"[{'confidence': 0.8772270083427429, 'joy': Lik...","[{'color': red: 231 green: 232 blue: 230 , 'sc...",[vans],[{'text': 'ista STERLY E DE INVERNO VANS OFF T...
3,C:\Users\smades2\Downloads\Image_Captioning\co...,"[Person, Tableware, Kitchen Appliance, Top, Gl...","[0.857225239276886, 0.7692965269088745, 0.7340...","[Tableware, Cup, Serveware, Drink, Cooking, En...","[0.8464375734329224, 0.8377450108528137, 0.800...",[],[],"[{'confidence': 0.640642523765564, 'joy': Like...","[{'color': red: 179 green: 116 blue: 48 , 'sco...",[],"[{'text': 'IIIII'}, {'text': 'IIIII'}]"
4,C:\Users\smades2\Downloads\Image_Captioning\co...,"[Pants, Hat, Necklace, Person, Shoe, Glasses, ...","[0.9430245161056519, 0.904980480670929, 0.8789...","[Shoe, Shoulder, Leg, Comfort, Knee, Thigh, Ey...","[0.9642908573150635, 0.9381376504898071, 0.926...",[],[],"[{'confidence': 0.8813260793685913, 'joy': Lik...","[{'color': red: 23 green: 19 blue: 24 , 'score...",[gentleman],"[{'text': '7'}, {'text': '7'}]"
5,C:\Users\smades2\Downloads\Image_Captioning\co...,"[Person, Person, Chair, Chair, Chair, Top, Stool]","[0.8972333073616028, 0.7690261602401733, 0.768...","[Furniture, Table, Chair, Wood, Leisure, Windo...","[0.9531746506690979, 0.8186572790145874, 0.802...",[],[],"[{'confidence': 0.43760138750076294, 'joy': Li...","[{'color': red: 196 green: 194 blue: 188 , 'sc...",[table],"[{'text': '7.00 874'}, {'text': '7.00'}, {'tex..."
6,C:\Users\smades2\Downloads\Image_Captioning\co...,"[Drink, Flowerpot, Houseplant]","[0.8427062630653381, 0.8346221446990967, 0.737...","[Plant, Tableware, Espressino, Drinkware, Flow...","[0.9514379501342773, 0.9500962495803833, 0.942...",[],[],[],"[{'color': red: 203 green: 195 blue: 188 , 'sc...",[flowerpot],[]
7,C:\Users\smades2\Downloads\Image_Captioning\co...,"[Laptop, Computer keyboard, Packaged goods, Food]","[0.8798336386680603, 0.7382544279098511, 0.703...","[Tableware, Table, Furniture, Drinkware, Plant...","[0.9626955986022949, 0.9512333273887634, 0.936...",[Assemblea Nacional Catalana],[0.6042346954345703],[],"[{'color': red: 31 green: 25 blue: 24 , 'score...",[table],[{'text': '7 essential JJJ 0 0 P return shift ...


In [17]:
## Checking individual columns

df_image.logo_description[1]

['Pagani']

#  TRIALS: SINGLE APIs

In [8]:
############################### Single File analysis using Image URL ############################


# image = types.Image()
# image.source.image_uri = 'http://pbs.twimg.com/media/FgZ_Y1cUoAEpdHM.jpg'

In [11]:
#### LABEL DETECTION ######

response_label = client.label_detection(image=image)

for label in response_label.label_annotations:
    print({'label': label.description, 'score': label.score})

{'label': 'Smile', 'score': 0.968029797077179}
{'label': 'Cap', 'score': 0.8980864882469177}
{'label': 'Vision care', 'score': 0.8734223246574402}
{'label': 'Coffee cup sleeve', 'score': 0.8734127879142761}
{'label': 'Goggles', 'score': 0.8692046403884888}
{'label': 'Eyewear', 'score': 0.8681694865226746}
{'label': 'Sunglasses', 'score': 0.8484448194503784}
{'label': 'Cool', 'score': 0.8132413625717163}
{'label': 'Baseball cap', 'score': 0.799035906791687}
{'label': 'Selfie', 'score': 0.7399507761001587}


In [12]:
#### LOGO DETECTION ######

response_logo = client.logo_detection(image=image)

for logo in response_logo.logo_annotations:
    print({'logo': logo.description, 'score': logo.score})

{'logo': 'Dallas Cowboys', 'score': 0.7527015805244446}


In [13]:
#### OBJECT DETECTION (LOCALIZATION) #####


response_object = client.object_localization(image=image).localized_object_annotations

print('Number of objects found: {}'.format(len(response_object)))

for object_ in response_object:
    print('\n{} (confidence: {})'.format(object_.name, object_.score))

Number of objects found: 4

Hat (confidence: 0.9370125532150269)

Person (confidence: 0.8221161961555481)

Sunglasses (confidence: 0.7236596941947937)

Packaged goods (confidence: 0.6298435926437378)


In [14]:
#### FACE DETECTION ######

response_face = client.face_detection(image=image)

face_data = []

for face_detection in response_face.face_annotations:
    d = {
        'confidence': face_detection.detection_confidence,
        'joy': face_detection.joy_likelihood,
        'sorrow': face_detection.sorrow_likelihood,
        'surprise': face_detection.surprise_likelihood,
        'anger': face_detection.anger_likelihood
    }
    print(d)

{'confidence': 0.8807474970817566, 'joy': <Likelihood.VERY_LIKELY: 5>, 'sorrow': <Likelihood.VERY_UNLIKELY: 1>, 'surprise': <Likelihood.VERY_UNLIKELY: 1>, 'anger': <Likelihood.VERY_UNLIKELY: 1>}


In [15]:
#### IMAGE PROPERTIES ######

response_image = client.image_properties(image=image)

image_data = []

for c in response_image.image_properties_annotation.dominant_colors.colors[:3]:
    d = {
        'color': c.color,
        'score': c.score,
        'pixel_fraction': c.pixel_fraction
    }
    print(d)

{'color': red: 61
green: 49
blue: 27
, 'score': 0.10590069741010666, 'pixel_fraction': 0.0745132714509964}
{'color': red: 115
green: 120
blue: 122
, 'score': 0.07405231893062592, 'pixel_fraction': 0.09935103356838226}
{'color': red: 185
green: 192
blue: 197
, 'score': 0.07207569479942322, 'pixel_fraction': 0.08997049927711487}


In [16]:
#### BEST LABEL DETECTION ######

response_best_label = client.web_detection(image=image)
annotations = response_best_label.web_detection

if annotations.best_guess_labels:
    for label in annotations.best_guess_labels:
        print('\nBest guess label: {}'.format(label.label))



Best guess label: cap


In [17]:
#### TEXT DETECTION ######

response_text = client.text_detection(image=image)

for r in response_text.text_annotations:
    d = {
        'text': r.description
    }
    print(d)

{'text': "DECAF\nItem: 1 of 1\nItems in order: 1\nJonathan'\nDo not m\nA&F\nCOWBOYS\nEst.1960"}
{'text': 'DECAF'}
{'text': 'Item'}
{'text': ':'}
{'text': '1'}
{'text': 'of'}
{'text': '1'}
{'text': 'Items'}
{'text': 'in'}
{'text': 'order'}
{'text': ':'}
{'text': '1'}
{'text': 'Jonathan'}
{'text': "'"}
{'text': 'Do'}
{'text': 'not'}
{'text': 'm'}
{'text': 'A'}
{'text': '&'}
{'text': 'F'}
{'text': 'COWBOYS'}
{'text': 'Est.1960'}


In [24]:
# ########### SINGLE API #################


# image = types.Image(content=im)
# #### OBJECT DETECTION (LOCALIZATION) #####


# response_object = client.object_localization(image=image).localized_object_annotations
# object_name,object_score=[], []
# print('Number of objects found: {}'.format(len(response_object)))

# for object_ in response_object:
#     print('\n{} (confidence: {})'.format(object_.name, object_.score))
#     object_name.append(object_.name)
#     object_score.append(object_.score)
    
# df = df.append({'object_name':object_name,'B':object_score},ignore_index=True )

Number of objects found: 7

Sunglasses (confidence: 0.9149388074874878)

Coffee cup (confidence: 0.8950873613357544)

Saucer (confidence: 0.8575760126113892)

Person (confidence: 0.8451477885246277)

Necklace (confidence: 0.6903539896011353)

Cat (confidence: 0.5961508750915527)

Drink (confidence: 0.5157045125961304)
